In [1]:
import numpy as np

# 1. Simulate data

$$y_t = u'_t \phi + x'_t \beta + \epsilon_t, \epsilon_t \stackrel{\text{i.i.d}}{\sim} \mathcal{N}(0, \sigma^{2}) $$

In [6]:
l = 0
k = 100
T = 200

In [7]:
a, b, A, B = 1, 1, 1, 1

### Simulate X

In [3]:
from scipy.linalg import toeplitz

In [4]:
rho = 0.75

In [27]:
def draw_X(rho=rho, k=k, T=T):
    correlation_matrix = toeplitz(rho ** np.arange(0, k))
    return np.random.multivariate_normal(np.zeros(k), correlation_matrix, size=T)

### Simulate U

### Simulate Y

In [63]:
def draw_Y(U, X, phi, beta, sigma_squared):
    return U @ phi + X @ beta + np.random.normal(0, sigma_squared, size=(X @ beta).shape[0])

# 2. Draw from posteriors

### (I). Draw from $R^2$

### (II). Draw from $\phi$

In [67]:
def inverse_matrix(A):
    try:
        return np.linalg.solve(A, np.eye(A.shape[0]))
    except:
        return A

In [68]:
def draw_phi(U, Y, X, beta, sigma_squared):
    U_U_inv = inverse_matrix(U.T @ U)
    mean_phi = U_U_inv @ U.T @ (Y - X @ beta)
    var_phi = sigma_squared * U_U_inv
    return np.random.multivariate_normal(mean_phi, var_phi, size=1)

### (III). Draw from z

In [88]:
def draw_tildes(Y, U, X, phi, beta, gamma, z):
    X_tilde = X[:, z != 0]
    beta_tilde = beta[z != 0]
    W_tilde = X_tilde.T @ X_tilde + (1/gamma**2) * np.eye(np.sum(z))
    Y_tilde = Y - U @ phi
    beta_tilde_hat = inverse_matrix(W_tilde) @ X_tilde.T @ Y_tilde
    return X_tilde, beta_tilde, W_tilde, Y_tilde, beta_tilde_hat

### (IV). Draw from $\sigma^2$

In [69]:
import scipy.stats as stats

In [89]:
def draw_sigma(Y, U, X, phi, beta, gamma, z):
    X_tilde, beta_tilde, W_tilde, Y_tilde, beta_tilde_hat = draw_tildes(Y, U, X, phi, beta, gamma, z)
    variance = (1/2) * (Y_tilde.T @ Y_tilde - beta_tilde_hat.T @ W_tilde @ beta_tilde_hat)
    return stats.invgamma.rvs(T/2, variance, size=1)

### (V). Draw from $\tilde{\beta}$

In [ ]:
def draw_beta_tilde(Y, U, X, phi, beta, gamma, z, sigma_squared):
    X_tilde, beta_tilde, W_tilde, Y_tilde, beta_tilde_hat = draw_tildes(Y, U, X, phi, beta, gamma, z)
    mean = inverse_matrix(W_tilde) @ X_tilde.T @ Y_tilde
    variance = sigma_squared * inverse_matrix(W_tilde)